In [1]:
import pygame
import pygame_gui
import random
from math import cos, sin, pi
from tkinter import Tk, colorchooser

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 800, 600

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)

# Font settings
font_size = 15
font = pygame.font.SysFont("monospace", font_size)

# ASCII Patterns
HEART_PATTERN = ["  **   **  ", " ***** ***** ", "***********", " ********* ", "  *******  ", "   *****   ", "    ***    ", "     *     "]
SMILEY_PATTERN = ["  *****  ", " *     * ", "* *   * *", "*       *", "* *   * *", "*  ***  *", " *     * ", "  *****  "]
ARROW_PATTERN = ["    *    ", "   ***   ", "  *****  ", " ******* ", "*********", " ******* ", "  *****  ", "   ***   ", "    *    "]
PATTERNS = [HEART_PATTERN, SMILEY_PATTERN, ARROW_PATTERN]

def float_range(start, stop, step):
    r = start
    while r < stop:
        yield r
        r += step

# Define the Symbol class
class Symbol:
    def __init__(self, x, y, speed):
        self.x = x
        self.y = y
        self.speed = speed
        self.switch_interval = random.randint(2, 25)
        self.color = GREEN  # Initial color
        self.character_set = "numbers"  # Initial character set
        self.value = self.random_value()  # Initial random value

    def random_value(self):
        if self.character_set == "numbers":
            return str(random.randint(0, 9))
        elif self.character_set == "letters":
            return random.choice("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")
        elif self.character_set == "special":
            return random.choice("!@#$%^&*()-_=+[]{}|;:,.<>?/")
        else:  # Patterns
            pattern = random.choice(PATTERNS)
            row = random.randint(0, len(pattern) - 1)
            col = random.randint(0, len(pattern[row]) - 1)
            char = pattern[row][col]
            while char == " ":  # Ensure we don't pick a space
                row = random.randint(0, len(pattern) - 1)
                col = random.randint(0, len(pattern[row]) - 1)
                char = pattern[row][col]
            return char

    def set_random_symbol(self, interval):
        if pygame.time.get_ticks() % interval == 0:
            self.value = self.random_value()

    def move(self):
        self.y += self.speed
        if self.y > HEIGHT:
            self.y = 0 - font_size
            self.value = self.random_value()

    def draw(self, screen):
        text = font.render(self.value, True, self.color)
        screen.blit(text, (self.x, self.y))

# Define the Stream class
class Stream:
    def __init__(self, x, y):
        self.symbols = []
        self.num_symbols = random.randint(5, 30)
        self.speed = random.randint(5, 20)
        for i in range(self.num_symbols):
            symbol = Symbol(x, y - i * font_size, self.speed)
            self.symbols.append(symbol)

    def draw(self, screen, interval):
        for symbol in self.symbols:
            symbol.draw(screen)
            symbol.move()
            symbol.set_random_symbol(interval)

    def set_speed(self, speed):
        for symbol in self.symbols:
            symbol.speed = speed        

# Define the EnhancedRotatingShape class
class EnhancedRotatingShape:
    def __init__(self, x, y, size):
        self.x = x
        self.y = y
        self.size = size
        self.angle = 0
        self.mode = "lines"  # Initial mode; can be "lines" or "characters"
        self.shape_type = "cube"  # Initial shape; can be "cube", "pyramid", and more
        self.color = GREEN  # Initial color of the shape
        self.switch_interval = 15  # Initial switch interval
        self.current_char = Symbol(0, 0, 0).random_value()  # Initialize current character

    def get_projected_points(self):
        points = []
        if self.shape_type == "cube":
            for i in [-1, 1]:
                for j in [-1, 1]:
                    for k in [-1, 1]:
                        x = self.x + (i * self.size * cos(self.angle) - k * self.size * sin(self.angle))
                        y = self.y + (j * self.size + i * self.size * sin(self.angle) + k * self.size * cos(self.angle))
                        points.append((x, y))
        elif self.shape_type == "pyramid":
            points.append((self.x, self.y - self.size))
            for i in [-1, 1]:
                for j in [-1, 1]:
                    x = self.x + i * self.size * cos(self.angle)
                    y = self.y + j * self.size * sin(self.angle) + i * self.size * cos(self.angle)
                    points.append((x, y))
        elif self.shape_type == "sphere":
            for i in range(-180, 180, 15):
                for j in range(-90, 90, 10):
                    x = self.x + self.size * cos(j) * cos(i)
                    y = self.y + self.size * cos(j) * sin(i)
                    points.append((x, y))
        elif self.shape_type == "star":
            scale = self.size * 2
            for i in range(5):
                angle = i * 2 * pi / 5 - pi / 2
                x = self.x + scale * cos(angle)
                y = self.y + scale * sin(angle)
                points.append((x, y))
        elif self.shape_type == "spiral":
            a, b, c = self.x, self.size / 10, self.y
            for t in float_range(0, 10 * pi, 0.1):  # 10 turns of spiral
                x = a + b * t * cos(t)
                y = c + b * t * sin(t)
                points.append((x, y))
        return points

    def draw_as_lines(self, screen):
        line_thickness = 1
        points = self.get_projected_points()
        if self.shape_type == "cube":
            for i in range(4):
                pygame.draw.line(screen, self.color, points[i], points[(i+1)%4], line_thickness)
                pygame.draw.line(screen, self.color, points[i+4], points[(i+1)%4+4], 1)
                pygame.draw.line(screen, self.color, points[i], points[i+4], 1)
        elif self.shape_type == "pyramid":
            for i in range(1, 4):
                pygame.draw.line(screen, self.color, points[0], points[i], 1)
                pygame.draw.line(screen, self.color, points[i], points[i % 3 + 1], 1)
        elif self.shape_type == "sphere":
            for i in range(0, len(points), 18):
                for j in range(18):
                    pygame.draw.line(screen, self.color, points[i + j], points[i + (j+1)%18], 1)
                if i < len(points) - 18:
                    for j in range(18):
                        pygame.draw.line(screen, self.color, points[i + j], points[i + j + 18], 1)
        elif self.shape_type == "star":
            for i in range(5):
                pygame.draw.line(screen, self.color, points[i], points[(i + 2) % 5], line_thickness)
        elif self.shape_type == "spiral":
            for i in range(len(points) - 1):
                pygame.draw.line(screen, self.color, points[i], points[i+1], 1)

    def draw_as_characters(self, screen):
        line_thickness = int(shape_font_size_slider.get_current_value() / 15)
        points = self.get_projected_points()
        if pygame.time.get_ticks() % self.switch_interval == 0:
            char = Symbol(0, 0, 0).random_value()  # Only change char when it's time to switch
        else:
            char = self.current_char  # Use the current character otherwise

        if self.shape_type == "cube":
            for i in range(4):
                self.draw_characters_between_points(screen, points[i], points[(i+1)%4], char)
                self.draw_characters_between_points(screen, points[i+4], points[(i+1)%4+4], char)
                self.draw_characters_between_points(screen, points[i], points[i+4], char)
        elif self.shape_type == "pyramid":
            for i in range(1, 4):
                self.draw_characters_between_points(screen, points[0], points[i], char)
                self.draw_characters_between_points(screen, points[i], points[i % 3 + 1], char)
        elif self.shape_type == "sphere":
            for i in range(0, len(points), 18):
                for j in range(18):
                    self.draw_characters_between_points(screen, points[i + j], points[i + (j+1)%18], char)
                if i < len(points) - 18:
                    for j in range(18):
                        self.draw_characters_between_points(screen, points[i + j], points[i + j + 18], char)
        elif self.shape_type == "star":
            for i in range(5):
                self.draw_characters_between_points(screen, points[i], points[(i + 2) % 5], char)
        elif self.shape_type == "spiral":
            for point in points:
                text = shape_font.render(char, True, self.color)
                screen.blit(pygame.transform.scale(text, (shape_font_size, shape_font_size)), point)

        self.current_char = char  # Update the current character

    def draw_characters_between_points(self, screen, start, end, char):
        # Given two points, draw characters between them
        length = int(((end[0] - start[0])**2 + (end[1] - start[1])**2) ** 0.5)
        for i in range(length // font_size):
            ratio = i / (length // font_size)
            x = start[0] + ratio * (end[0] - start[0])
            y = start[1] + ratio * (end[1] - start[1])
            screen.blit(shape_font.render(char, True, self.color), (x, y))

    def draw(self, screen):
        if self.mode == "lines":
            self.draw_as_lines(screen)
        elif self.mode == "characters":
            self.draw_as_characters(screen)

    def update(self):
        self.angle += 0.01

    def toggle_mode(self):
        modes = ["lines", "characters"]
        idx = modes.index(self.mode)
        self.mode = modes[(idx + 1) % len(modes)]

    def move(self, direction):
        if direction == "UP":
            self.y -= 10
        elif direction == "DOWN":
            self.y += 10
        elif direction == "LEFT":
            self.x -= 10
        elif direction == "RIGHT":
            self.x += 10

    def set_shape(self, shape_type):
        self.shape_type = shape_type

    def set_color(self, color):
        self.color = color


# Setup the display
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Matrix Rain Animation Player")

# Create streams and the rotating shape
shape = EnhancedRotatingShape(WIDTH // 2, HEIGHT // 2, 100)
streams = [Stream(x, random.randint(-800, 0)) for x in range(0, WIDTH, font_size)]
bg_color = BLACK
shape_font_size = 15
shape_font = pygame.font.SysFont("monospace", shape_font_size)
rain_active = True

# GUI Setup
manager = pygame_gui.UIManager((WIDTH, HEIGHT), "theme.json")

# Dropdown for character selector
character_selector = pygame_gui.elements.UIDropDownMenu(
    options_list=['Numbers', 'Letters', 'Special', 'Pattern'],
    starting_option='Numbers',
    relative_rect=pygame.Rect((10, 10), (150, 30)),
    manager=manager
)

# Dropdown for shape selector
shape_selector = pygame_gui.elements.UIDropDownMenu(
    options_list=['Cube', 'Pyramid', 'Sphere', 'Star', 'Spiral'],
    starting_option='Cube',
    relative_rect=pygame.Rect((10, 50), (150, 30)),
    manager=manager
)

# Buttons for toggling rain, toggling mode, and changing colors
toggle_rain_button = pygame_gui.elements.UIButton(
    relative_rect=pygame.Rect((10, 90), (150, 30)),
    text='Toggle Rain',
    manager=manager
)
toggle_mode_button = pygame_gui.elements.UIButton(
    relative_rect=pygame.Rect((10, 130), (150, 30)),
    text='Toggle Mode',
    manager=manager
)
change_rain_color_button = pygame_gui.elements.UIButton(
    relative_rect=pygame.Rect((10, 170), (150, 30)),
    text='Rain Color',
    manager=manager
)
change_shape_color_button = pygame_gui.elements.UIButton(
    relative_rect=pygame.Rect((10, 210), (150, 30)),
    text='Shape Color',
    manager=manager
)
change_bg_color_button = pygame_gui.elements.UIButton(
    relative_rect=pygame.Rect((10, 250), (150, 30)),
    text='Background Color',
    manager=manager
)

# Sliders for rain speed, symbol change speed, and shape size
rain_speed_label = pygame_gui.elements.UILabel(
    relative_rect=pygame.Rect((10, 290), (150, 20)),
    text='Rain Speed',
    manager=manager
)
rain_speed_slider = pygame_gui.elements.UIHorizontalSlider(
    relative_rect=pygame.Rect((10, 310), (150, 20)),
    start_value=10,
    value_range=(1, 30),
    manager=manager
)

rain_symbol_change_label = pygame_gui.elements.UILabel(
    relative_rect=pygame.Rect((10, 340), (150, 20)),
    text='Rain Char. Speed',
    manager=manager
)
rain_symbol_change_slider = pygame_gui.elements.UIHorizontalSlider(
    relative_rect=pygame.Rect((10, 360), (150, 20)),
    start_value=35,  # Changed the start value for a more noticeable effect
    value_range=(50, 2),  # Inverted the range
    manager=manager
)

shape_symbol_change_label = pygame_gui.elements.UILabel(
    relative_rect=pygame.Rect((10, 390), (150, 20)),
    text='Shape Char. Speed',
    manager=manager
)
shape_symbol_change_slider = pygame_gui.elements.UIHorizontalSlider(
    relative_rect=pygame.Rect((10, 410), (150, 20)),
    start_value=35,  # Changed the start value for a more noticeable effect
    value_range=(50, 2),  # Inverted the range
    manager=manager
)

shape_size_label = pygame_gui.elements.UILabel(
    relative_rect=pygame.Rect((10, 440), (150, 20)),
    text='Shape Size',
    manager=manager
)
shape_size_slider = pygame_gui.elements.UIHorizontalSlider(
    relative_rect=pygame.Rect((10, 460), (150, 20)),
    start_value=100,
    value_range=(50, 200),
    manager=manager
)

# Sliders for rain and shape font size
rain_font_size_label = pygame_gui.elements.UILabel(
    relative_rect=pygame.Rect((10, 490), (150, 20)),
    text='Rain Font Size',
    manager=manager
)
rain_font_size_slider = pygame_gui.elements.UIHorizontalSlider(
    relative_rect=pygame.Rect((10, 510), (150, 20)),
    start_value=15,  # Initial value
    value_range=(10, 30),  # Font size range
    manager=manager
)

shape_font_size_label = pygame_gui.elements.UILabel(
    relative_rect=pygame.Rect((10, 540), (150, 20)),
    text='Shape Font Size',
    manager=manager
)
shape_font_size_slider = pygame_gui.elements.UIHorizontalSlider(
    relative_rect=pygame.Rect((10, 560), (150, 20)),
    start_value=15,  # Initial value
    value_range=(10, 30),  # Font size range
    manager=manager
)


# Main loop for the animation
def matrix_rain_animation():
    global streams, shape, bg_color, rain_active, font_size, font
    global rain_active, bg_color
    running = True
    clock = pygame.time.Clock()
    while running:
        time_delta = clock.tick(60)/1000.0
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    shape.move("UP")
                elif event.key == pygame.K_DOWN:
                    shape.move("DOWN")
                elif event.key == pygame.K_LEFT:
                    shape.move("LEFT")
                elif event.key == pygame.K_RIGHT:
                    shape.move("RIGHT")

            if event.type == pygame.USEREVENT:
                if event.user_type == pygame_gui.UI_DROP_DOWN_MENU_CHANGED:
                    if event.ui_element == character_selector:
                        selected_option = character_selector.selected_option
                        if selected_option == "Numbers":
                            for stream in streams:
                                for symbol in stream.symbols:
                                    symbol.character_set = "numbers"
                        elif selected_option == "Letters":
                            for stream in streams:
                                for symbol in stream.symbols:
                                    symbol.character_set = "letters"
                        elif selected_option == "Special":
                            for stream in streams:
                                for symbol in stream.symbols:
                                    symbol.character_set = "special"
                        elif selected_option == "Pattern":
                            for stream in streams:
                                for symbol in stream.symbols:
                                    symbol.character_set = "pattern"
                    elif event.ui_element == shape_selector:
                        selected_option = shape_selector.selected_option
                        if selected_option == "Cube":
                            shape.set_shape("cube")
                        elif selected_option == "Pyramid":
                            shape.set_shape("pyramid")
                        elif selected_option == "Sphere":
                            shape.set_shape("sphere")
                        elif selected_option == "Star":
                            shape.set_shape("star")
                        elif selected_option == "Spiral":
                            shape.set_shape("spiral")

                if event.user_type == pygame_gui.UI_BUTTON_PRESSED:
                    if event.ui_element == toggle_rain_button:
                        rain_active = not rain_active
                    elif event.ui_element == toggle_mode_button:
                        shape.toggle_mode()
                    elif event.ui_element == change_rain_color_button:
                        root = Tk()
                        root.withdraw()  # Hide the main window
                        color_code = colorchooser.askcolor(title="Choose rain color")
                        root.destroy()
                        if color_code and color_code[0]:
                            for stream in streams:
                                for symbol in stream.symbols:
                                    symbol.color = color_code[0]
                    elif event.ui_element == change_shape_color_button:
                        root = Tk()
                        root.withdraw()  # Hide the main window
                        color_code = colorchooser.askcolor(title="Choose shape color")
                        root.destroy()
                        if color_code and color_code[0]:
                            shape.set_color(color_code[0])
                    elif event.ui_element == change_bg_color_button:
                        root = Tk()
                        root.withdraw()  # Hide the main window
                        color_code = colorchooser.askcolor(title="Choose background color")
                        root.destroy()
                        if color_code and color_code[0]:
                            bg_color = color_code[0]

            manager.process_events(event)

        for stream in streams:
            stream.set_speed(rain_speed_slider.get_current_value())

        global font_size, font
        global shape_font_size, shape_font
        current_rain_font_size = font_size
        if current_rain_font_size != int(rain_font_size_slider.get_current_value()):
            font_size = int(rain_font_size_slider.get_current_value())
            font = pygame.font.SysFont("monospace", font_size)
            streams = [Stream(x, random.randint(-800, 0)) for x in range(0, WIDTH, font_size)]
        current_shape_font_size = shape_font_size
        if current_shape_font_size != int(shape_font_size_slider.get_current_value()):
            shape_font_size = int(shape_font_size_slider.get_current_value())
            shape_font = pygame.font.SysFont("monospace", shape_font_size)

        screen.fill(bg_color)

        if rain_active:
            for stream in streams:
                stream.draw(screen, int(rain_symbol_change_slider.get_current_value()))

        shape.switch_interval = int(shape_symbol_change_slider.get_current_value())
        shape.size = int(shape_size_slider.get_current_value())
        if shape.shape_type == "spiral" and shape.size >= 200:
            shape.size = 50
        shape.update()
        shape.draw(screen)

        manager.update(time_delta)
        manager.draw_ui(screen)

        pygame.display.flip()

    pygame.quit()

# Run the animation
matrix_rain_animation()

pygame-ce 2.3.1 (SDL 2.26.5, Python 3.10.8)


C:\Users\tessi\AppData\Roaming\Python\Python310\site-packages\pygame_gui\core\ui_appearance_theme.py:668: UserWarning: Failed to open theme file at path:theme.json
  warnings.warn("Failed to open theme file at path:" + str(file_path))
C:\Users\tessi\AppData\Local\Temp\ipykernel_19796\3868122872.py:402: DeprecationWarning: Pygame GUI event types can now be used directly as event.type rather than event.user_type. This old style user_type event will go away in version 0.8.0
  if event.user_type == pygame_gui.UI_DROP_DOWN_MENU_CHANGED:
C:\Users\tessi\AppData\Local\Temp\ipykernel_19796\3868122872.py:434: DeprecationWarning: Pygame GUI event types can now be used directly as event.type rather than event.user_type. This old style user_type event will go away in version 0.8.0
  if event.user_type == pygame_gui.UI_BUTTON_PRESSED:
